In [19]:
import pandas as pd
import numpy as np
import os
import re

# 데이터 불러오기

In [32]:
df1 = pd.read_csv('../raw/선수정보_2018.csv')
df2 = pd.read_csv('../raw/선수정보_2019.csv')
df3 = pd.read_csv('../raw/선수정보_2020.csv')
df4 = pd.read_csv('../raw/선수정보_2021.csv')

In [81]:
#각 타구별로 선수이름과 배럴타구여부를 붙인 데이터
df5 = pd.read_csv('../../dataset/배럴타구여부추가.csv')
df5.head()

,PCODE,GYEAR,G_ID,PIT_ID,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM,NAME,barrel,Date
0,60100,2018,20180328HHNC0,180328_203348,HH,7,148.19,-4.7,땅볼아웃,140.56,마산,백창수,False,20180328
1,60100,2018,20180330SKHH0,180330_190656,HH,2,153.79,5.8,직선타,142.41,대전,백창수,False,20180330
2,60100,2018,20180331SKHH0,180331_174531,HH,2,148.65,-13.5,땅볼아웃,135.50,대전,백창수,False,20180331
3,60100,2018,20180331SKHH0,180331_192505,HH,7,151.85,-3.7,병살타,131.71,대전,백창수,False,20180331
4,60100,2018,20180331SKHH0,180331_200628,HH,9,154.12,19.0,플라이,141.85,대전,백창수,True,20180331


In [34]:
#연도정보 추가 및 concat 수행
df1['year'] = 2018
df2['year'] = 2019
df3['year'] = 2020
df4['year'] = 2021

In [35]:
df = pd.concat([df1,df2,df3,df4]).reset_index().drop('index', axis=1)
df.head()

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA,year
0,정훈,04-29,한화,W,4:3,7,3B,1,2,0,...,1,0.000,0.000,0.000,0.000,8,1.51,-0.72,-0.046,2018
1,정훈,05-02,KIA,L,6:12,6,1B,1,4,0,...,0,0.125,0.111,0.250,0.361,12,1.09,-0.33,-0.032,2018
2,정훈,05-04,@SK,L,4:5,9,PH,0,1,1,...,0,0.300,0.273,0.800,1.073,3,0.44,1.00,0.025,2018
3,정훈,05-08,@LG,W,4:2,5,PH,0,1,0,...,0,0.333,0.308,0.750,1.058,6,0.49,-0.26,-0.013,2018
4,정훈,05-10,@LG,W,7:2,9,PH,0,1,0,...,0,0.333,0.313,0.667,0.979,3,0.02,-0.26,-0.001,2018


In [36]:
#Merge를 위해 Date 정보 추가
df['날짜'] = df['날짜'].str.replace("-", "")
df['Date'] = df['year'].astype('str') + df['날짜']

In [55]:
#동명이인이 존재하므로 해당선수에 대해 구분할 수 있게 정보 추가
df[df['이름'].apply(lambda x : bool(re.match(string = x, pattern='[a-zA-Z]')))]['이름'].unique()

array(['WO김재현', 'SK김재현', 'SS김민수', 'LT김민수'], dtype=object)

In [62]:
wo_jae_index = df5[(df5['NAME'] == '김재현') & (df5['T_ID'] == 'WO')].index
sk_jae_index = df5[(df5['NAME'] == '김재현') & (df5['T_ID'] == 'SK')].index
ss_min_index = df5[(df5['NAME'] == '김민수') & (df5['T_ID'] == 'SS')].index
lt_min_index = df5[(df5['NAME'] == '김민수') & (df5['T_ID'] == 'LT')].index

In [82]:
df5.loc[wo_jae_index, 'NAME'] = 'WO김재현'
df5.loc[sk_jae_index, 'NAME'] = 'SK김재현'
df5.loc[ss_min_index, 'NAME'] = 'SS김민수'
df5.loc[lt_min_index, 'NAME'] = 'LT김민수'

In [83]:
#선수별/날짜별 배럴타구 생산 개수 계산
df5 = df5[['PCODE', 'NAME', 'barrel', 'Date']]
df5['Date'] = df5['Date'].astype('str')

In [84]:
df5_barrel = df5.groupby(['PCODE', 'NAME','Date']).sum().reset_index()
df5_barrel

,PCODE,NAME,Date,barrel
0,50054,천성호,20200603,0
1,50054,천성호,20200604,0
2,50054,천성호,20200607,0
3,50054,천성호,20200611,0
4,50054,천성호,20200613,0
...,...,...,...,...
49665,99810,이진영,20181006,0
49666,99810,이진영,20181009,0
49667,99810,이진영,20181010,1
49668,99810,이진영,20181012,2


In [103]:
#결측치를 채우기 위해 {PCODE : 이름} dict 생성 
pcode_dict = dict(df5.drop_duplicates(subset=['PCODE', 'NAME']).iloc[:, :2].values)

In [90]:
#날짜별 배럴타구 개수 데이터를 기존 월별 선수 성적 데이터에 MERGE
df_merge = df.rename(columns={'이름':'NAME'}).merge(df5_barrel, how='left')

In [106]:
#MERGE시에 해당 경기 타구가 없는 선수의 경우 PCODE가 누락되므로 채워줌
for pcode, name in pcode_dict.items():
    idx = df_merge[df_merge['NAME'] == name].index
    df_merge.loc[idx, "PCODE"] = pcode

In [108]:
#마찬가지로 해당 경기에 타구가 없다면 barrel 타구가 none이 들어가므로 0으로 채우기
df_merge['barrel'] = df_merge['barrel'].fillna(0)

In [109]:
df_merge

,NAME,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,장타,OPS,투구,avLI,RE24,WPA,year,Date,PCODE,barrel
0,정훈,0429,한화,W,4:3,7,3B,1,2,0,...,0.000,0.000,8,1.51,-0.72,-0.046,2018,20180429,60523.0,0.0
1,정훈,0502,KIA,L,6:12,6,1B,1,4,0,...,0.250,0.361,12,1.09,-0.33,-0.032,2018,20180502,60523.0,0.0
2,정훈,0504,@SK,L,4:5,9,PH,0,1,1,...,0.800,1.073,3,0.44,1.00,0.025,2018,20180504,60523.0,1.0
3,정훈,0508,@LG,W,4:2,5,PH,0,1,0,...,0.750,1.058,6,0.49,-0.26,-0.013,2018,20180508,60523.0,0.0
4,정훈,0510,@LG,W,7:2,9,PH,0,1,0,...,0.667,0.979,3,0.02,-0.26,-0.001,2018,20180510,60523.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63452,김상수,0829,@KT,L,3:8,9,2B,1,3,0,...,0.265,0.577,18,0.50,0.66,0.046,2021,20210829,79402.0,0.0
63453,김상수,0901,키움,W,7:1,9,2B,1,2,2,...,0.268,0.588,18,0.67,2.07,0.098,2021,20210901,79402.0,0.0
63454,김상수,0903,@KIA,W,9:3,6,2B,1,3,1,...,0.268,0.590,11,0.57,0.90,0.060,2021,20210903,79402.0,0.0
63455,김상수,0905,두산,L,5:6,9,2B,1,3,0,...,0.267,0.591,15,1.78,-0.02,-0.057,2021,20210905,79402.0,0.0


In [112]:
#지성준 : 64717 한동민 : 62895 백민기 : 63559
#개명 선수의 경우 PCODE 상에 반영이 안되므로 따로 채워줌
df_merge[df_merge['PCODE'].isnull()]['NAME'].unique()

array(['지성준', '한동민', '백민기'], dtype=object)

In [127]:
for name, pcode in {"지성준" : 64717, "한동민" : 62895, "백민기" : 63559}.items():
    idx = df_merge[df_merge['NAME'] == name].index
    df_merge.loc[idx, "PCODE"] = pcode

In [129]:
df_merge.to_csv('../../dataset/total_baseball_PCODE&배럴타구추가.csv', index=False, 
                encoding='utf-8-sig')